<a href="https://colab.research.google.com/github/Raut-Raj/IBM_Projects/blob/main/capston_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

# Define helper functions
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(table_cells):
    return [i for i in table_cells.strings][0]

def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        new_mass = mass[0:mass.find("kg")+2]
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name = ' '.join(row.contents)
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

# URL for the Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Request the Falcon9 Launch Wiki page
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the table containing the launch records
html_tables = soup.find_all('table', "wikitable plainrowheaders collapsible")
first_launch_table = html_tables[2]

# Extract column names
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name:
        column_names.append(name)

# Create a dictionary to hold the data
launch_dict = {name: [] for name in column_names if name != "Date and time ( )"}
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Extract launch data
for rows in first_launch_table.find_all("tr"):
    if rows.th:
        flight_number = rows.th.string.strip()
        if flight_number.isdigit():
            row = rows.find_all('td')
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]
            bv = booster_version(row[1]) or row[1].a.string
            launch_site = row[2].a.string
            payload = row[3].a.string
            payload_mass = get_mass(row[4])
            orbit = row[5].a.string
            customer = row[6].a.string
            launch_outcome = list(row[7].strings)[0]
            booster_landing = landing_status(row[8])

            # Append data to the dictionary
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

# Convert dictionary to DataFrame
df = pd.DataFrame(launch_dict)

# Save to CSV
df.to_csv('spacex_web_scraped.csv', index=False)

# Display the first few rows of the DataFrame
print(df.head())


  Flight No.     Launch site                       Payload Payload mass Orbit  \
0         14  Cape Canaveral                  SpaceX CRS-5     2,395 kg   LEO   
1         15  Cape Canaveral                        DSCOVR       570 kg   HEO   
2         16  Cape Canaveral                        ABS-3A     4,159 kg   GTO   
3         17  Cape Canaveral                  SpaceX CRS-6     1,898 kg   LEO   
4         18  Cape Canaveral  TürkmenÄlem 52°E / MonacoSAT     4,707 kg   GTO   

  Customer Launch outcome Version Booster Booster landing              Date  \
0     NASA        Success        F9 v1.1[        Failure    10 January 2015   
1     USAF      Success\n        F9 v1.1[      Controlled  11 February 2015   
2      ABS      Success\n        F9 v1.1[      No attempt      2 March 2015   
3     NASA      Success\n        F9 v1.1[         Failure     14 April 2015   
4     None      Success\n        F9 v1.1[      No attempt     27 April 2015   

    Time  
0  09:47  
1  23:03  
2  03